In [36]:
import pandas as pd

In [37]:
df = pd.read_csv('https://raw.githubusercontent.com/VitorBonella/PL-Dataset/main/dataset.csv',sep=";") #Leitura dos dados para o pandas

In [38]:
df.set_index('id',inplace=True) #Transformando a coluna id no indice da tabela

#Observem que a classe esta separada em duas colunas então devemos concatenadas para formar uma coluna só chamada classe
df['classe'] = df['tipo_lampada'].str.replace(" ", "") + df['potencia'].astype(str) 

In [39]:
# Lista de Indices
HU = ['i1', 'i2', 'i3', 'i4','i5', 'i6', 'i7']
MY = HU

In [40]:
'''
Os dados dos descritores como podemos observar na tabela acima estão com ","
porém o padrão para ponto flutuante (float) em python é "." logo a biblioteca 
entendeu que se tratavam de strings (objects para o pandas) e não de floats 
assim é importante transformar esses dados para float.
'''
df[MY] = df[MY].apply(lambda x: x.str.replace(',', '.').astype(float), axis=1)

### Separando dados de teste e treino

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[MY], df['classe'], test_size=0.2, random_state=36851234)

## zeroR

In [44]:
from sklearn.model_selection import cross_val_predict 
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score


zR = DummyClassifier(strategy='most_frequent')

cv_zR = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores = cross_val_score(zR, X_train, y_train, scoring='accuracy', cv=cv_zR)

print(scores.mean())

0.15217391304347824
